In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.extra.rate_limiter import RateLimiter

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

In [2]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_national_capitals_by_population',header=0)[1]

In [3]:
df

,Rank,Country/Territory,Capital,Population,Year,% ofcountry'spopulation
0,1,China PR,Beijing,"21,542,000[1]",2010,1.5%
1,2,Japan,Tokyo,"13,929,286[2]",2017,11.03%
2,3,Russia,Moscow,"12,506,468[3]",2011,8.52%
3,4,DR Congo,Kinshasa,"11,855,000[4]",2012,12.9%
4,5,Indonesia,Jakarta,"10,075,310[5]",2011,3.76%
5,6,South Korea,Seoul,"9,838,892[6]",2015,19.03%
6,7,Egypt,Cairo,9500000,2012,9.54%
7,8,Mexico,Mexico City,"8,918,653[7]",2015,7.05%
8,9,Bangladesh,Dhaka,"8,906,039 [8]",2011,5.52%
9,10,United Kingdom England,London,"8,908,081[9]",2015,13.19%


In [5]:
df.to_excel("capitals.xlsx")

In [6]:
chg=[(57,"Guayaquil"),(81,"Guatemala"),(89,"Naypyidaw"),(100,"Jerusalem"),
     (154,"El Aaiún"),(163,"Berne"),(171,"Nicosia"),(200,"Ramallah"),(231,"Yaren"),(235,"Vatican City"),(239,"Brades")]

In [7]:
for i in range(len(chg)):
    df.loc[chg[i][0],"Capital"]=chg[i][1]
df.loc[154,"Population"]="194668"

In [8]:
df

,Rank,Country/Territory,Capital,Population,Year,% ofcountry'spopulation
0,1,China PR,Beijing,"21,542,000[1]",2010,1.5%
1,2,Japan,Tokyo,"13,929,286[2]",2017,11.03%
2,3,Russia,Moscow,"12,506,468[3]",2011,8.52%
3,4,DR Congo,Kinshasa,"11,855,000[4]",2012,12.9%
4,5,Indonesia,Jakarta,"10,075,310[5]",2011,3.76%
5,6,South Korea,Seoul,"9,838,892[6]",2015,19.03%
6,7,Egypt,Cairo,9500000,2012,9.54%
7,8,Mexico,Mexico City,"8,918,653[7]",2015,7.05%
8,9,Bangladesh,Dhaka,"8,906,039 [8]",2011,5.52%
9,10,United Kingdom England,London,"8,908,081[9]",2015,13.19%


In [9]:
coord=pd.DataFrame(columns=["Capital","Latitude","Longitude"])
err=pd.DataFrame(columns=["Number","Name"])

In [10]:
for i in df.index:
    geolocator = Nominatim(user_agent="tr_explorer")
    address =df.loc[i,"Capital"]    
    try:
        location = geolocator.geocode(address,timeout=1)
        latitude = location.latitude
        longitude = location.longitude        
    except:
        coord=coord.append({"Capital":address,"Latitude":np.nan,"Longitude":np.nan},ignore_index=True)
        err=err.append({"Number":i,"Name":address},ignore_index=True)
        print('The geograpical coordinate of {} denied'.format(address))
    else:
        coord=coord.append({"Capital":address,"Latitude":latitude,"Longitude":longitude},ignore_index=True)
        print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Beijing are 39.906217, 116.3912757.
The geograpical coordinate of Tokyo are 35.6828387, 139.7594549.
The geograpical coordinate of Moscow are 55.4792046, 37.3273304.
The geograpical coordinate of Kinshasa are -4.3217055, 15.3125974.
The geograpical coordinate of Jakarta are -6.1753942, 106.827183.
The geograpical coordinate of Seoul are 37.564982549999996, 126.93921080358436.
The geograpical coordinate of Cairo are 30.048819, 31.243666.
The geograpical coordinate of Mexico City are 19.4326296, -99.1331785.
The geograpical coordinate of Dhaka are 23.7593572, 90.3788136.
The geograpical coordinate of London are 51.5073219, -0.1276474.
The geograpical coordinate of Lima are -12.0621065, -77.0365256.
The geograpical coordinate of Tehran are 35.7006177, 51.4013785.
The geograpical coordinate of Bangkok are 13.7542529, 100.493087.
The geograpical coordinate of Hanoi are 21.0294498, 105.8544441.
The geograpical coordinate of Riyadh are 24.6319692, 46.7150648.
The

The geograpical coordinate of Asunción are -25.2800459, -57.6343814.
The geograpical coordinate of Macau are 22.1757605, 113.5514142.
The geograpical coordinate of Edinburgh are 55.9533456, -3.1883749.
The geograpical coordinate of Djibouti (city) are 11.8145966, 42.8453061.
The geograpical coordinate of Yamoussoukro are 6.809107, -5.273263.
The geograpical coordinate of Bissau are 11.861324, -15.583055.
The geograpical coordinate of Tallinn are 59.4372155, 24.7453688.
The geograpical coordinate of Bratislava are 48.1516988, 17.1093063.
The geograpical coordinate of San Juan are 18.465299, -66.116666.
The geograpical coordinate of Canberra are -35.2975906, 149.1012676.
The geograpical coordinate of Wellington are -41.2887953, 174.7772114.
The geograpical coordinate of Sarajevo are 43.8519774, 18.3866868.
The geograpical coordinate of Juba are 4.8472017, 31.5951655.
The geograpical coordinate of Cardiff are 51.4816546, -3.1791934.
The geograpical coordinate of Belfast are 54.5964411, -5

In [12]:
err

,Number,Name
0,58,Harare
1,67,Prague
2,77,Freetown
3,87,Ottawa
4,96,Zagreb
5,105,N'Djamena
6,163,Berne
7,173,Mbabane
8,182,Port of Spain
9,191,Tarawa


In [13]:
for i in err.index:
    geolocator = Nominatim(user_agent="tr_explorer")
    address =err.loc[i,"Name"]
    try:
        location = geolocator.geocode(address,timeout=1)
        latitude = location.latitude
        longitude = location.longitude        
    except:
        print('The geograpical coordinate of {} denied'.format(address))
    else:
        coord.loc[err.loc[i,"Number"],"Latitude"]=latitude
        coord.loc[err.loc[i,"Number"],"Longitude"]=longitude
        err=err.drop(i,axis=0)
        print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Harare are -17.831773, 31.045686.
The geograpical coordinate of Prague are 50.0874654, 14.4212535.
The geograpical coordinate of Freetown are 8.479004, -13.26795.
The geograpical coordinate of Ottawa are 45.421106, -75.690308.
The geograpical coordinate of Zagreb are 45.813177, 15.977048.
The geograpical coordinate of N'Djamena are 12.1191543, 15.0502758.
The geograpical coordinate of Berne are 46.9482713, 7.4514512.
The geograpical coordinate of Mbabane are -26.325745, 31.144663.
The geograpical coordinate of Port of Spain are 10.6572678, -61.5180173.
The geograpical coordinate of Tarawa are 11.5884149, 3.9707743.
The geograpical coordinate of Majuro are 7.0909924, 171.3816354.
The geograpical coordinate of Basseterre are 17.2960919, -62.722301.
The geograpical coordinate of Avarua are -21.2074736, -159.7708145.
The geograpical coordinate of The Valley are 41.5595723, -98.9805484.
The geograpical coordinate of Brades are 16.7890601, -62.208731.


In [14]:
err

,Number,Name


In [13]:
    address ="Vatican City"
    geolocator = Nominatim(user_agent="yudin_vd")
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))
    except:
        print('The geograpical coordinate of {} denied'.format(address))

The geograpical coordinate of Vatican City are 41.90381485, 12.453152716779822.


In [15]:
coord

,Capital,Latitude,Longitude
0,Beijing,39.906217,116.391276
1,Tokyo,35.682839,139.759455
2,Moscow,55.479205,37.327330
3,Kinshasa,-4.321706,15.312597
4,Jakarta,-6.175394,106.827183
5,Seoul,37.564983,126.939211
6,Cairo,30.048819,31.243666
7,Mexico City,19.432630,-99.133178
8,Dhaka,23.759357,90.378814
9,London,51.507322,-0.127647


In [16]:
df["Latitude"]=coord["Latitude"]
df["Longitude"]=coord["Longitude"]

In [17]:
df

,Rank,Country/Territory,Capital,Population,Year,% ofcountry'spopulation,Latitude,Longitude
0,1,China PR,Beijing,"21,542,000[1]",2010,1.5%,39.906217,116.391276
1,2,Japan,Tokyo,"13,929,286[2]",2017,11.03%,35.682839,139.759455
2,3,Russia,Moscow,"12,506,468[3]",2011,8.52%,55.479205,37.327330
3,4,DR Congo,Kinshasa,"11,855,000[4]",2012,12.9%,-4.321706,15.312597
4,5,Indonesia,Jakarta,"10,075,310[5]",2011,3.76%,-6.175394,106.827183
5,6,South Korea,Seoul,"9,838,892[6]",2015,19.03%,37.564983,126.939211
6,7,Egypt,Cairo,9500000,2012,9.54%,30.048819,31.243666
7,8,Mexico,Mexico City,"8,918,653[7]",2015,7.05%,19.432630,-99.133178
8,9,Bangladesh,Dhaka,"8,906,039 [8]",2011,5.52%,23.759357,90.378814
9,10,United Kingdom England,London,"8,908,081[9]",2015,13.19%,51.507322,-0.127647


In [18]:
for i in coord.index:
    if coord.loc[i,"Capital"]!=df.loc[i,"Capital"]:
        print("Ошибка в {} строке".format(i))

In [19]:
for i in df.index:
    b=df.loc[i,"Population"].find("[")
    if b!=-1:
        df.loc[i,"Population"]=df.loc[i,"Population"][:b]
    b=df.loc[i,"Population"].find("(")
    if b!=-1:
        df.loc[i,"Population"]=df.loc[i,"Population"][:b-1]
    df.loc[i,"Population"]=df.loc[i,"Population"].replace(",","")
        

In [20]:
df[["Population"]]=df[["Population"]].astype("int")

In [23]:
df=df.drop("Rank",axis=1)

In [24]:
df

,Country/Territory,Capital,Population,Year,% ofcountry'spopulation,Latitude,Longitude
0,China PR,Beijing,21542000,2010,1.5%,39.906217,116.391276
1,Japan,Tokyo,13929286,2017,11.03%,35.682839,139.759455
2,Russia,Moscow,12506468,2011,8.52%,55.479205,37.327330
3,DR Congo,Kinshasa,11855000,2012,12.9%,-4.321706,15.312597
4,Indonesia,Jakarta,10075310,2011,3.76%,-6.175394,106.827183
5,South Korea,Seoul,9838892,2015,19.03%,37.564983,126.939211
6,Egypt,Cairo,9500000,2012,9.54%,30.048819,31.243666
7,Mexico,Mexico City,8918653,2015,7.05%,19.432630,-99.133178
8,Bangladesh,Dhaka,8906039,2011,5.52%,23.759357,90.378814
9,United Kingdom England,London,8908081,2015,13.19%,51.507322,-0.127647


In [28]:
df["CountVenue"]=df["Population"]//100000*5

In [30]:
for i in df.index:
    if df.loc[i,"Population"]<2000000:
        df.loc[i,"CountVenue"]=100

In [31]:
df

,Country/Territory,Capital,Population,Year,% ofcountry'spopulation,Latitude,Longitude,CountVenue
0,China PR,Beijing,21542000,2010,1.5%,39.906217,116.391276,1075
1,Japan,Tokyo,13929286,2017,11.03%,35.682839,139.759455,695
2,Russia,Moscow,12506468,2011,8.52%,55.479205,37.327330,625
3,DR Congo,Kinshasa,11855000,2012,12.9%,-4.321706,15.312597,590
4,Indonesia,Jakarta,10075310,2011,3.76%,-6.175394,106.827183,500
5,South Korea,Seoul,9838892,2015,19.03%,37.564983,126.939211,490
6,Egypt,Cairo,9500000,2012,9.54%,30.048819,31.243666,475
7,Mexico,Mexico City,8918653,2015,7.05%,19.432630,-99.133178,445
8,Bangladesh,Dhaka,8906039,2011,5.52%,23.759357,90.378814,445
9,United Kingdom England,London,8908081,2015,13.19%,51.507322,-0.127647,445


In [32]:
df["CountVenue"].sum()

32880

In [34]:
df.to_excel("capitals_2.xlsx", index=False)